In [3]:
import warnings
warnings.filterwarnings("ignore")  # Ignore all warnings

#import Lib 
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


In [4]:
    # 1. Load CSV and clean
df = pd.read_csv("RELIANCE_StockData.csv")
df.rename(columns={'Price': 'Date'}, inplace=True)
# df = df.iloc[2:].reset_index(drop=True)
df['Date'] = pd.to_datetime(df['Date'])
df = df.dropna()

In [5]:
# 2. Select numeric columns
feature_names = ['Close', 'High', 'Low', 'Open', 'Volume']
data = df[feature_names].values


In [6]:
# 3. Scale data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

In [7]:
# 4. Create sequences for LSTM
def create_sequences(dataset, sequence_length=60):
    X, y = [], []
    for i in range(len(dataset) - sequence_length):
        X.append(dataset[i:i+sequence_length])
        y.append(dataset[i+sequence_length])
    return np.array(X), np.array(y)

sequence_length = 60
X, y = create_sequences(scaled_data, sequence_length)


In [8]:
# 5. Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [9]:
# 6. Simple LSTM model
model = Sequential([
    LSTM(50, dropout=0.2, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(len(feature_names))
])

model.compile(optimizer='adam', loss='mse')


In [10]:
# 7. Train model
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)


In [11]:
# 8. Predictions
predictions = model.predict(X_test, verbose=0)
predictions = scaler.inverse_transform(predictions)
y_test_original = scaler.inverse_transform(y_test)


In [ ]:
# 9. visualization
fig, axes = plt.subplots(3, 2, figsize=(14, 10))
axes = axes.flatten()
for i, feature in enumerate(feature_names):
    axes[i].plot(y_test_original[:, i], label=f'Actual {feature}')
    axes[i].plot(predictions[:, i], label=f'Predicted {feature}', linestyle='--')
    axes[i].set_title(f'{feature} - Actual vs Predicted')
    axes[i].grid(True)
    axes[i].legend()

if len(feature_names) < len(axes):
    axes[-1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# 10. predict next day's values
def predict_next_day(last_n_days):
    """
    last_n_days: np.array of shape (sequence_length, num_features)
    """
    if last_n_days.shape != (sequence_length, len(feature_names)):
        raise ValueError(f"Input must have shape ({sequence_length}, {len(feature_names)})")
    
    scaled_input = scaler.transform(last_n_days)
    scaled_input = np.expand_dims(scaled_input, axis=0)  # reshape for LSTM
    scaled_prediction = model.predict(scaled_input, verbose=0)
    return scaler.inverse_transform(scaled_prediction)[0]


last_sequence = data[-sequence_length:] 
next_day_pred = predict_next_day(last_sequence)
print("\nNext Day Prediction:")
for f, val in zip(feature_names, next_day_pred):
    print(f"{f}: {val:.2f}")


In [ ]:
from datetime import datetime, timedelta

# Predict N future days ahead
def predict_until_date(target_date_str):
    target_date = datetime.strptime(target_date_str, "%Y-%m-%d").date()
    last_known_date = df['Date'].iloc[-1].date()

    if target_date <= last_known_date:
        raise ValueError("Target date must be after the dataset's last date.")

    current_sequence = data[-sequence_length:].copy()
    current_date = last_known_date

    prediction_for_target = None

    while current_date < target_date:
        # Predict next day
        pred = predict_next_day(current_sequence)

        # Append prediction to sequence and drop first row
        current_sequence = np.vstack([current_sequence[1:], pred])
        
        current_date += timedelta(days=1)
        
        if current_date == target_date:
            prediction_for_target = pred

    return current_date, prediction_for_target


# Example: User input date
user_input_date = input("Enter target date (YYYY-MM-DD): ").strip()
pred_date, pred_values = predict_until_date(user_input_date)

print(f"\nPredicted values for {pred_date}:")
for f, val in zip(feature_names, pred_values):
    print(f"{f}: {val:.2f}")



Predicted values for 2025-08-12:
Close: 1252.18
High: 1257.26
Low: 1237.73
Open: 1235.53
Volume: 13856888.00
